# 📘 Task Explanation: Shared Memory and Bank Conflicts in CUDA

## 🎯 Objective
The objective of this task is to understand how **CUDA shared memory** works, why it is significantly faster than global memory, and how **bank conflicts** can degrade its performance.  
You will then implement a **shared-memory tile copy kernel** to practice using shared memory correctly and efficiently.

This task introduces one of the most important performance concepts in CUDA and GPU programming.

---

## 🧠 Background: What Is Shared Memory?
Shared memory is a **small, on-chip memory** that is:
- Shared by all threads within the same thread block
- Much faster than global memory
- Manually managed by the programmer

Typical characteristics:
- Low latency (similar to registers)
- Limited size (e.g., 48–64 KB per SM)
- Lifetime = one thread block

Shared memory is widely used in:
- Matrix multiplication (tiling)
- Reductions
- Convolutions
- Normalization layers (LayerNorm, BatchNorm)
- Attention kernels

---

## 🧩 Part A — Understand Bank Conflicts

### What Are Memory Banks?
Shared memory is divided into multiple **banks** (typically 32 banks).  
Each bank can service **one access per cycle**.

- If threads in a warp access **different banks** → no conflict  
- If multiple threads access the **same bank** → **bank conflict**

### Why Bank Conflicts Matter
Bank conflicts cause:
- Serialized memory accesses
- Increased latency
- Lower effective bandwidth

Even though shared memory is fast, **poor access patterns can make it slow**.

---

## 🔍 Example Concept (No Code Required)
You should be able to reason about:
- Why accessing `shared[threadIdx.x]` is conflict-free
- Why accessing `shared[threadIdx.x * stride]` may cause conflicts
- How padding shared memory can eliminate conflicts

---

## 🧩 Part B — Implement a Shared-Memory Tile Copy Kernel

### Task
Write a CUDA kernel that:
1. Loads a **tile** of data from global memory into shared memory  
2. Synchronizes threads within the block  
3. Writes the tile back to global memory  

The kernel should:
- Use shared memory as an intermediate buffer
- Correctly synchronize threads using `__syncthreads()`
- Avoid out-of-bounds memory access

---

## 🧠 Why a Tile Copy Kernel?
A tile copy kernel is a **minimal example** that demonstrates:
- Correct shared memory usage
- Thread cooperation within a block
- The impact of access patterns on shared memory performance

This pattern is a building block for:
- Matrix multiplication
- Transpose kernels
- Convolution kernels
- ML operators using tiling

---

## ⚠️ Common Pitfalls to Watch For
- Forgetting `__syncthreads()` after loading shared memory  
- Accessing shared memory with patterns that cause bank conflicts  
- Using incorrect shared memory indexing  
- Reading/writing out of bounds  

---

## 🧪 Deliverables
You should produce:
1. A CUDA kernel that copies data using shared memory tiling  
2. A correctness check comparing input and output  
3. (Optional) A benchmark comparing:
   - Direct global memory copy  
   - Shared-memory tile copy  

---

## 🎓 What You Learn from This Task
By completing this task, you will understand:
- How shared memory differs from global memory
- How bank conflicts arise and why they hurt performance
- How to design shared-memory access patterns
- Why padding and layout matter in GPU kernels

These skills are essential for:
- CUDA optimization
- ML kernel development
- Writing high-performance GPU code
- Understanding libraries like cuBLAS, cuDNN, and Triton

---

## 🚀 Relevance to ML Systems
Many ML kernels rely heavily on shared memory:
- Matrix multiply uses tiled shared memory
- LayerNorm uses shared memory for reductions
- Attention kernels reuse data via shared memory

Mastering shared memory and bank conflicts is a **core skill for ML Systems Engineers and GPU Kernel Engineers**.

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Fri Dec 26 21:49:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8       

In [ ]:
!apt-get update
!apt-get install -y cuda-toolkit-12-4

In [ ]:
%%writefile shared_mem_tile_copy_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

// ------------------------------------------------------------
// TODO (Task): Shared-memory tile copy kernel
// Goal:
//  - Copy input array A -> output array B using shared memory tiles.
// Requirements:
//  - Each block loads a tile from global memory into shared memory.
//  - Synchronize threads (barrier) before writing back.
//  - Write the tile from shared memory to global memory.
//  - Avoid out-of-bounds accesses.
// Notes:
//  - Start with 1D tiling (simpler) or extend to 2D tiling (optional).
// ------------------------------------------------------------
__global__ void tileCopyShared(const float* __restrict__ A,
                               float* __restrict__ B,
                               int N) {
    // TODO: declare shared memory tile
    // Example (static): __shared__ float tile[TILE_SIZE];
    // Or (dynamic): extern __shared__ float tile[];
    extern __shared__ float tile[];

    // TODO: compute global index
    // int tid = ...
    // int gid = ...
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    int tid = threadIdx.x;

    // TODO: load from global -> shared (guarded)
    // if (...) tile[...] = A[...];
    if(tid < N){
      tile[tid] = A[gid];
    }

    // TODO: synchronize
    // __syncthreads();
    __syncthreads();

    // TODO: store from shared -> global (guarded)
    // if (...) B[...] = tile[...];
    if (gid < N){
        B[gid] = tile[tid];
    }
}

// ------------------------------------------------------------
// CPU reference copy
// ------------------------------------------------------------
static void copyCPU(const float* A, float* B, int N) {
    for (int i = 0; i < N; ++i) B[i] = A[i];
}

// ------------------------------------------------------------
// TODO: correctness checker
// Requirements:
//  - Compare arrays elementwise within tolerance
//  - Print first mismatch and return false
// ------------------------------------------------------------
static bool checkClose(const float* out, const float* ref, int N, float tol) {
    // TODO
    for(int i = 0; i < N; ++i){
        float diff = fabsf(out[i] - ref[i]);
        if (diff > tol){
          return false;
        }
    }
    return true;
}

int main() {
    const int N = 1 << 24; // ~16M floats
    const float tol = 1e-6f;
    const size_t bytes = size_t(N) * sizeof(float);

    // Host alloc
    float* hA = (float*)std::malloc(bytes);
    float* hB = (float*)std::malloc(bytes);
    float* hRef = (float*)std::malloc(bytes);

    if (!hA || !hB || !hRef) {
        fprintf(stderr, "Host malloc failed.\n");
        return EXIT_FAILURE;
    }

    // Init input
    for (int i = 0; i < N; ++i) hA[i] = 0.001f * i;

    // CPU reference
    copyCPU(hA, hRef, N);

    // Device alloc
    float *dA = nullptr, *dB = nullptr;
    CUDA_CHECK(cudaMalloc(&dA, bytes));
    CUDA_CHECK(cudaMalloc(&dB, bytes));

    // H2D
    CUDA_CHECK(cudaMemcpy(dA, hA, bytes, cudaMemcpyHostToDevice));

    // ------------------------------------------------------------
    // TODO: Choose launch configuration
    // - Decide TILE_SIZE (or blockSize)
    // - Decide gridSize
    // - If using dynamic shared memory, pass sharedMemBytes
    // ------------------------------------------------------------
    int blockSize = 256;     // TODO
    int gridSize  = (N + blockSize - 1) / blockSize;     // TODO
    size_t sharedMemBytes = blockSize * sizeof(float); // TODO (0 if using static shared mem)

    // Launch
    tileCopyShared<<<gridSize, blockSize, sharedMemBytes>>>(dA, dB, N);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    // D2H
    CUDA_CHECK(cudaMemcpy(hB, dB, bytes, cudaMemcpyDeviceToHost));

    // ------------------------------------------------------------
    // TODO: Correctness check
    // ------------------------------------------------------------
    bool ok = checkClose(hB, hRef, N, tol);
    printf("Correctness: %s\n", ok ? "PASS" : "FAIL");

    // Cleanup
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    std::free(hA);
    std::free(hB);
    std::free(hRef);

    return ok ? EXIT_SUCCESS : EXIT_FAILURE;
}

Overwriting shared_mem_tile_copy_skeleton.cu


In [ ]:
!nvcc -arch=sm_75 shared_mem_tile_copy_skeleton.cu -o tile_copy
!./tile_copy

Correctness: PASS


# 📘 Task Explanation: Shared Memory Bank-Conflict Experiment

## 🎯 Objective
The objective of this task is to **understand, observe, and quantify shared-memory bank conflicts** in CUDA.  
You will design controlled experiments that access shared memory with different patterns and measure how **bank conflicts impact kernel performance**.

This task builds intuition about why shared memory can be either **extremely fast or unexpectedly slow**, depending on access patterns.

---

## 🧠 Background: What Is a Bank Conflict?
Shared memory on NVIDIA GPUs is divided into **multiple memory banks** (typically 32 banks).

- Each bank can service **one access per cycle**
- Threads in a **warp (32 threads)** access shared memory simultaneously

### Access Scenarios
- **Conflict-free**: each thread accesses a different bank  
- **Bank conflict**: two or more threads access the same bank  
- **Result**: accesses are serialized → increased latency

Even though shared memory is on-chip, **bank conflicts can significantly degrade performance**.

---

## 🧩 Part A — Design Conflict-Free Shared Memory Access

### Task
Create a kernel where each thread accesses shared memory in a **conflict-free pattern**, for example:

- Thread `t` accesses `shared[t]`
- Consecutive threads map to consecutive banks

### Goal
Establish a **performance baseline** where shared memory is used optimally.

---

## 🧩 Part B — Introduce Bank Conflicts via Strided Access

### Task
Modify the kernel so that threads access shared memory with a **stride**, such as:

- Thread `t` accesses `shared[t * stride]`

Test multiple stride values (e.g., 1, 2, 4, 8).

### Expected Effect
As the stride increases:
- More threads map to the same bank
- The number of bank conflicts increases
- Kernel execution time increases

---

## 🧩 Part C — Eliminate Bank Conflicts Using Padding

### Task
Redesign the shared memory layout by adding **padding**, such as:

- Declaring shared memory with extra elements
- Changing indexing so threads map to different banks

### Goal
Demonstrate that **bank conflicts are a layout problem**, not an inherent limitation of shared memory.

---

## 📊 What to Measure
For each configuration (conflict-free, conflicted, padded):

- Kernel execution time
- (Optional) Effective shared memory bandwidth
- Relative slowdown compared to conflict-free case

---

## 📈 Expected Observations

| Access Pattern | Bank Conflicts | Performance |
|---------------|---------------|-------------|
| Contiguous (`shared[t]`) | None | Fastest |
| Strided (`shared[t*2]`) | Moderate | Slower |
| Highly strided (`shared[t*4]`) | Severe | Much slower |
| Padded layout | Eliminated | Near baseline |

---

## 🔍 Key Questions to Answer
- How does stride affect bank mapping?
- Why does padding eliminate conflicts?
- Why do bank conflicts serialize memory access?
- How do these effects scale with block size?

---

## 🧪 Deliverables
You should produce:

1. A CUDA kernel demonstrating **conflict-free shared memory access**
2. A CUDA kernel demonstrating **bank-conflicted access**
3. A CUDA kernel demonstrating **conflict elimination via padding**
4. Benchmark results comparing all three cases
5. A short written analysis explaining:
   - How bank conflicts arise
   - Why padding works
   - How performance changes across configurations

---

## 🎓 What You Learn from This Task
By completing this task, you will understand:

- How shared memory banks work
- Why access patterns matter more than raw computation
- How to diagnose shared memory performance issues
- How to design bank-conflict-free shared memory layouts

These skills are critical for:
- CUDA optimization
- Matrix multiplication and transpose kernels
- Reductions and prefix sums
- ML kernels using shared memory (LayerNorm, Attention)

---

## 🚀 Relevance to ML Systems
Many high-performance ML kernels rely heavily on shared memory:

- GEMM tiling in cuBLAS
- LayerNorm and BatchNorm reductions
- Attention and FlashAttention kernels

Understanding and eliminating bank conflicts is a **core competency for ML Systems Engineers and GPU Kernel Engineers**.

This task trains the same reasoning used in real-world CUDA and Triton kernel optimization.

In [ ]:
%%writefile bank_conflict_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

__global__ void bankConflictKernel(float* out, int N, int mode, int stride, int iters) {
    extern __shared__ float smem[];

    int gid  = blockIdx.x * blockDim.x + threadIdx.x;
    int lane = threadIdx.x;

    // init: one element per thread for mode 0/1
    if (lane < blockDim.x) smem[lane] = (float)lane;
    __syncthreads();

    float acc = 0.0f;

    for (int k = 0; k < iters; ++k) {
        if (mode == 0) {
            // baseline
            int idx = lane;
            acc += smem[idx];

        } else if (mode == 1) {
            // strided (wrap to avoid OOB)
            int idx = (lane * stride) % blockDim.x;
            acc += smem[idx];

        } else {
            // padded tile experiment using only first 32 threads
            // to keep shared memory small and demonstrate bank behavior clearly
            const int TILE = 32;
            const int pitch = TILE + 1; // padding

            if (lane < TILE) {
                // layout uses first TILE*pitch floats in smem
                // treat as tile[row][col]
                int row = lane;
                int col = (k * stride) % TILE;

                int idx = row * pitch + col;  // within TILE*(TILE+1)
                acc += smem[idx];
            }
        }
    }

    if (gid < N) out[gid] = acc;
}

static float timeKernelMs(float* dOut, int N, int mode, int stride, int iters,
                          int gridSize, int blockSize, size_t sharedBytes,
                          int warmup, int runs) {
    for (int i = 0; i < warmup; ++i) {
        bankConflictKernel<<<gridSize, blockSize, sharedBytes>>>(dOut, N, mode, stride, iters);
    }
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    CUDA_CHECK(cudaEventRecord(start));
    for (int i = 0; i < runs; ++i) {
        bankConflictKernel<<<gridSize, blockSize, sharedBytes>>>(dOut, N, mode, stride, iters);
    }
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));
    return ms / runs;
}

int main() {
    const int N = 1 << 20;
    const int blockSize = 256;
    const int gridSize  = (N + blockSize - 1) / blockSize;

    const int strides[] = {1, 2, 4, 8, 16, 32};
    const int numStrides = (int)(sizeof(strides) / sizeof(strides[0]));

    const int iters  = 2048;
    const int warmup = 5;
    const int runs   = 20;

    // shared memory bytes:
    // need at least blockSize floats for mode 0/1
    // and at least TILE*(TILE+1) floats for mode 2
    const int TILE = 32;
    size_t needFloats = (size_t)blockSize;
    size_t paddedFloats = (size_t)TILE * (size_t)(TILE + 1);
    if (paddedFloats > needFloats) needFloats = paddedFloats;
    size_t sharedBytes = needFloats * sizeof(float);

    float* dOut = nullptr;
    CUDA_CHECK(cudaMalloc(&dOut, (size_t)N * sizeof(float)));

    printf("=== Bank Conflict Experiment (fixed) ===\n");
    printf("N=%d, grid=%d, block=%d, iters=%d, sharedBytes=%.2f KB\n",
           N, gridSize, blockSize, iters, sharedBytes / 1024.0);

    for (int s = 0; s < numStrides; ++s) {
        int stride = strides[s];

        for (int mode = 0; mode <= 2; ++mode) {
            float ms = timeKernelMs(dOut, N, mode, stride, iters,
                                    gridSize, blockSize, sharedBytes,
                                    warmup, runs);

            const char* name =
                (mode == 0) ? "baseline" :
                (mode == 1) ? "strided"  :
                              "padded32x33";

            printf("[stride=%2d][mode=%d:%s] time=%.4f ms\n", stride, mode, name, ms);
        }
        printf("\n");
    }

    CUDA_CHECK(cudaFree(dOut));
    return 0;
}

Overwriting bank_conflict_skeleton.cu


In [ ]:
!nvcc -arch=sm_75 bank_conflict_skeleton.cu -o bank_conflict_skeleton
!./bank_conflict_skeleton

=== Bank Conflict Experiment (fixed) ===
N=1048576, grid=4096, block=256, iters=2048, sharedBytes=4.12 KB
[stride= 1][mode=0:baseline] time=1.4614 ms
[stride= 1][mode=1:strided] time=1.5553 ms
[stride= 1][mode=2:padded32x33] time=5.3495 ms

[stride= 2][mode=0:baseline] time=0.9506 ms
[stride= 2][mode=1:strided] time=1.0115 ms
[stride= 2][mode=2:padded32x33] time=3.8302 ms

[stride= 4][mode=0:baseline] time=0.5552 ms
[stride= 4][mode=1:strided] time=0.6016 ms
[stride= 4][mode=2:padded32x33] time=3.0060 ms

[stride= 8][mode=0:baseline] time=0.5602 ms
[stride= 8][mode=1:strided] time=0.6188 ms
[stride= 8][mode=2:padded32x33] time=2.9954 ms

[stride=16][mode=0:baseline] time=0.5604 ms
[stride=16][mode=1:strided] time=0.6213 ms
[stride=16][mode=2:padded32x33] time=2.9825 ms

[stride=32][mode=0:baseline] time=0.5602 ms
[stride=32][mode=1:strided] time=0.6209 ms
[stride=32][mode=2:padded32x33] time=3.0003 ms



In [ ]:
%%writefile bank_conflict_clean.cu
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

// blockDim should be (32, 8) or (32, 16)
__global__ void bankConflictClean(float* out, int iters, int mode) {
    // We'll use a 32x32 tile. For padding mode, pitch=33.
    extern __shared__ float smem[];

    int tx = threadIdx.x; // 0..31
    int ty = threadIdx.y; // 0..(BLOCK_ROWS-1)

    const int TILE = 32;
    const int pitch = (mode == 2) ? (TILE + 1) : TILE; // padding only in mode=2

    // Initialize: each thread writes multiple rows (like transpose)
    for (int j = 0; j < TILE; j += blockDim.y) {
        int row = ty + j;
        smem[row * pitch + tx] = (float)(row * 0.1f + tx);
    }
    __syncthreads();

    float acc = 0.0f;

    for (int k = 0; k < iters; ++k) {
        // choose a column that changes over time (avoid cache weirdness)
        int col = (k & 31);

        int idx;
        if (mode == 0) {
            // Row access (conflict-free-ish): smem[ty][tx]
            idx = ty * pitch + tx;
        } else {
            // Column access (classic conflict when pitch=32):
            // smem[tx][col] -> threads in warp differ in row (tx), same col
            idx = tx * pitch + col;
        }
        acc += smem[idx];
    }

    // write something
    int gid = (blockIdx.x * blockDim.x * blockDim.y) + (ty * blockDim.x + tx);
    out[gid] = acc;
}

static float timeMs(float* dOut, int iters, int mode,
                    dim3 grid, dim3 block, size_t shBytes, int warmup, int runs) {
    for (int i=0;i<warmup;i++) bankConflictClean<<<grid, block, shBytes>>>(dOut, iters, mode);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    cudaEvent_t s,e;
    CUDA_CHECK(cudaEventCreate(&s));
    CUDA_CHECK(cudaEventCreate(&e));
    CUDA_CHECK(cudaEventRecord(s));
    for (int i=0;i<runs;i++) bankConflictClean<<<grid, block, shBytes>>>(dOut, iters, mode);
    CUDA_CHECK(cudaEventRecord(e));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(e));

    float ms=0;
    CUDA_CHECK(cudaEventElapsedTime(&ms,s,e));
    CUDA_CHECK(cudaEventDestroy(s));
    CUDA_CHECK(cudaEventDestroy(e));
    return ms/runs;
}

int main() {
    const int iters = 4096;
    const int warmup=5, runs=20;

    dim3 block(32, 8);
    dim3 grid(256);

    // Allocate enough for the padded case: 32*(33) floats
    size_t shBytes = 32 * 33 * sizeof(float);

    float* dOut=nullptr;
    CUDA_CHECK(cudaMalloc(&dOut, grid.x * block.x * block.y * sizeof(float)));

    printf("=== Clean Bank Conflict Demo ===\n");
    printf("block=(%d,%d) grid=%d iters=%d shmem=%.2fKB\n",
           block.x, block.y, (int)grid.x, iters, shBytes/1024.0);

    // mode 0: row access
    float t0 = timeMs(dOut, iters, 0, grid, block, shBytes, warmup, runs);

    // mode 1: column access with pitch=32 -> conflicts
    float t1 = timeMs(dOut, iters, 1, grid, block, shBytes, warmup, runs);

    // mode 2: column access with pitch=33 -> padding removes conflicts
    float t2 = timeMs(dOut, iters, 2, grid, block, shBytes, warmup, runs);

    printf("mode0 row(no conflict) : %.4f ms\n", t0);
    printf("mode1 col(pitch=32)    : %.4f ms (conflict)\n", t1);
    printf("mode2 col(pitch=33)    : %.4f ms (padded)\n", t2);

    CUDA_CHECK(cudaFree(dOut));
    return 0;
}


Writing bank_conflict_clean.cu


In [ ]:
!nvcc -arch=sm_75 bank_conflict_clean.cu -o bank_conflict_clean
!./bank_conflict_clean

=== Clean Bank Conflict Demo ===
block=(32,8) grid=256 iters=4096 shmem=4.12KB
mode0 row(no conflict) : 0.2050 ms
mode1 col(pitch=32)    : 12.6755 ms (conflict)
mode2 col(pitch=33)    : 0.2935 ms (padded)


# 📘 Follow-Up Task Explanation: Shared-Memory Matrix Multiplication (Tiled GEMM)

## 🎯 Objective
The objective of this task is to implement **matrix multiplication (GEMM)** using **shared memory tiling** in CUDA and understand **why shared memory dramatically improves performance** over a naive global-memory implementation.

This task builds directly on:
- Shared memory fundamentals
- Thread cooperation within a block
- Bank-conflict awareness
- Memory coalescing concepts

It is one of the **most important CUDA optimization patterns** and a foundation for ML workloads.

---

## 🧠 Background: Why Matrix Multiplication Matters
Matrix multiplication is the computational core of:
- Neural network layers (Linear, Attention, MLP)
- Convolutions (after im2col)
- Scientific computing and simulations

A naive CUDA implementation is usually **memory-bound** because:
- Each matrix element is loaded repeatedly from global memory
- Global memory latency dominates execution time

Shared-memory tiling solves this problem by **reusing data**.

---

## 🧩 Part A — Naive Matrix Multiplication (Baseline)

### Task
Implement a naive CUDA matrix multiplication kernel:

\[
C_{ij} = \sum_k A_{ik} \cdot B_{kj}
\]

Where:
- Each thread computes **one element** of matrix `C`
- All reads come directly from **global memory**

### Goal
Establish a **performance baseline** to compare against the optimized version.

---

## 🧩 Part B — Shared-Memory Tiled Matrix Multiplication

### Task
Rewrite the kernel using **shared memory tiling**:

1. Divide matrices into square **tiles** (e.g., 16×16 or 32×32)
2. Each thread block:
   - Loads one tile of `A` and one tile of `B` into shared memory
3. Synchronize threads
4. Compute partial dot products using shared memory
5. Repeat for all tiles along the K dimension

### Key Requirements
- Use `__shared__` memory for tiles
- Use `__syncthreads()` correctly
- Ensure global memory loads are coalesced
- Avoid shared-memory bank conflicts

---

## 🧠 Why Tiling Works
Shared-memory tiling:
- Reduces global memory loads by **reuse**
- Converts many slow global reads into fast shared reads
- Increases arithmetic intensity (FLOPs per byte)

This is the core idea behind **cuBLAS**, **Tensor Cores**, and most ML kernels.

---

## 🧩 Part C — Tile Size and Performance Tuning

### Task
Experiment with different tile sizes:
- 16×16
- 32×32

Observe how tile size affects:
- Performance
- Occupancy
- Shared memory usage

### Key Trade-offs
- Larger tiles → more reuse, but higher shared memory usage
- Smaller tiles → lower shared memory, but less reuse

---

## 📊 What to Measure
For both naive and tiled kernels, measure:
- Kernel execution time
- Speedup of tiled version over naive
- (Optional) Effective FLOPs or bandwidth

---

## 📈 Expected Observations

| Kernel Type | Memory Behavior | Performance |
|------------|-----------------|-------------|
| Naive | Repeated global loads | Slow |
| Tiled (shared memory) | Data reuse in shared memory | Much faster |
| Tuned tile size | Balanced reuse & occupancy | Best |

A correct tiled implementation should achieve **multiple-X speedup** over the naive version.

---

## ⚠️ Common Pitfalls
- Forgetting `__syncthreads()` after loading tiles
- Incorrect indexing into shared memory
- Out-of-bounds accesses at matrix edges
- Bank conflicts in shared memory tiles
- Choosing tile sizes that exceed shared memory limits

---

## 🧪 Deliverables
You should produce:
1. A **naive CUDA matrix multiplication kernel**
2. A **shared-memory tiled matrix multiplication kernel**
3. Benchmark results comparing both implementations
4. A short analysis explaining:
   - Why the tiled version is faster
   - How shared memory reuse improves performance
   - How tile size affects efficiency

---

## 🎓 What You Learn from This Task
By completing this task, you will understand:
- How to design cooperative thread-block algorithms
- How shared memory enables data reuse
- How memory hierarchy affects performance
- How real ML kernels are structured internally

This task is a **cornerstone skill** for:
- CUDA optimization
- ML kernel engineering
- Understanding cuBLAS and Triton matmul
- LLM training and inference acceleration

---

## 🚀 Relevance to ML Systems
Shared-memory matrix multiplication is the conceptual basis for:
- cuBLAS GEMM
- Tensor Core operations
- Attention QKᵀ and QKV kernels
- Triton and compiler-generated GPU kernels

Mastering this task means you are thinking like a **GPU kernel engineer**, not just a CUDA user.

In [ ]:
%%writefile tiled_gemm_shared_skeleton.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

// ------------------------------------------------------------
// TODO (Task): Shared-memory tiled matrix multiplication (GEMM)
// Compute C = A * B
//
// Shapes (row-major):
//   A: M x K
//   B: K x N
//   C: M x N
//
// Requirements:
//  - Each thread computes one (row, col) element of C
//  - Use shared memory tiling for A and B
//  - Use __syncthreads() correctly
//  - Avoid out-of-bounds at edges (M,N,K may not be multiples of TILE)
//  - Coalesced global loads for tiles (best effort)
// ------------------------------------------------------------

// TODO: choose a tile size (e.g., 16 or 32)
#ifndef TILE
#define TILE 0  // TODO
#endif

__global__ void matmulTiledShared(const float* __restrict__ A,
                                  const float* __restrict__ B,
                                  float* __restrict__ C,
                                  int M, int N, int K) {
    // TODO: declare shared memory tiles
    // __shared__ float As[TILE][TILE];
    // __shared__ float Bs[TILE][TILE];

    // TODO: compute global row/col for this thread
    // int row = ...
    // int col = ...

    // TODO: accumulator
    // float acc = 0.0f;

    // TODO: loop over tiles along K dimension
    // for (int t = 0; t < ...; ++t) {
    //   - load A tile element into As (guarded)
    //   - load B tile element into Bs (guarded)
    //   - __syncthreads()
    //   - compute partial dot product over k in [0, TILE)
    //   - __syncthreads()
    // }

    // TODO: write C[row, col] (guarded)
}

// ------------------------------------------------------------
// CPU reference GEMM (for correctness)
// ------------------------------------------------------------
static void matmulCPU(const float* A, const float* B, float* C, int M, int N, int K) {
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            float acc = 0.0f;
            for (int k = 0; k < K; ++k) {
                acc += A[i * K + k] * B[k * N + j];
            }
            C[i * N + j] = acc;
        }
    }
}

// ------------------------------------------------------------
// TODO: correctness checker
// Requirements:
//  - Compare C_gpu vs C_ref within tolerance
//  - Print first mismatch (i, value_gpu, value_ref)
// ------------------------------------------------------------
static bool checkClose(const float* gpu, const float* ref, int count, float tol) {
    // TODO
    return false;
}

// ------------------------------------------------------------
// Timing helper (CUDA events) - implemented
// ------------------------------------------------------------
static float timeGemmMs(const float* dA, const float* dB, float* dC,
                        int M, int N, int K,
                        dim3 grid, dim3 block,
                        int warmup, int iters) {
    for (int i = 0; i < warmup; ++i) {
        matmulTiledShared<<<grid, block>>>(dA, dB, dC, M, N, K);
    }
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    CUDA_CHECK(cudaEventRecord(start));
    for (int i = 0; i < iters; ++i) {
        matmulTiledShared<<<grid, block>>>(dA, dB, dC, M, N, K);
    }
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));

    return ms / iters;
}

// ------------------------------------------------------------
// TODO (optional): estimate GFLOPS
// GEMM does ~2*M*N*K FLOPs
// GFLOPS = FLOPs / time_seconds / 1e9
// ------------------------------------------------------------
static double estimateGFLOPS(int M, int N, int K, float kernel_ms) {
    // TODO
    return 0.0;
}

int main() {
    // ------------------------------------------------------------
    // Sizes (you can tune these)
    // NOTE: try sizes that are NOT multiples of TILE to test edges.
    // ------------------------------------------------------------
    const int M = 1024;
    const int N = 1024;
    const int K = 1024;

    const float tol = 1e-3f; // TODO: choose tolerance (float accumulation)
    const size_t bytesA = size_t(M) * K * sizeof(float);
    const size_t bytesB = size_t(K) * N * sizeof(float);
    const size_t bytesC = size_t(M) * N * sizeof(float);

    // Host alloc
    float* hA = (float*)std::malloc(bytesA);
    float* hB = (float*)std::malloc(bytesB);
    float* hC_gpu = (float*)std::malloc(bytesC);
    float* hC_ref = (float*)std::malloc(bytesC);

    if (!hA || !hB || !hC_gpu || !hC_ref) {
        fprintf(stderr, "Host malloc failed.\n");
        return EXIT_FAILURE;
    }

    // Init A, B
    for (int i = 0; i < M * K; ++i) hA[i] = 0.001f * (i % 1000);
    for (int i = 0; i < K * N; ++i) hB[i] = 0.002f * (i % 1000);

    // CPU reference (can be slow for large sizes; reduce sizes if needed)
    matmulCPU(hA, hB, hC_ref, M, N, K);

    // Device alloc
    float *dA=nullptr, *dB=nullptr, *dC=nullptr;
    CUDA_CHECK(cudaMalloc(&dA, bytesA));
    CUDA_CHECK(cudaMalloc(&dB, bytesB));
    CUDA_CHECK(cudaMalloc(&dC, bytesC));

    CUDA_CHECK(cudaMemcpy(dA, hA, bytesA, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB, bytesB, cudaMemcpyHostToDevice));

    // ------------------------------------------------------------
    // TODO: Choose launch config based on TILE
    // Typical:
    //   block = (TILE, TILE)
    //   grid  = (ceil(N/TILE), ceil(M/TILE))
    // ------------------------------------------------------------
    dim3 block(0, 0, 1); // TODO
    dim3 grid(0, 0, 1);  // TODO

    // Launch once (correctness)
    matmulTiledShared<<<grid, block>>>(dA, dB, dC, M, N, K);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    CUDA_CHECK(cudaMemcpy(hC_gpu, dC, bytesC, cudaMemcpyDeviceToHost));

    // ------------------------------------------------------------
    // TODO: correctness check
    // ------------------------------------------------------------
    bool ok = checkClose(hC_gpu, hC_ref, M * N, tol);
    printf("Correctness: %s\n", ok ? "PASS" : "FAIL");

    // Timing
    const int warmup = 5;
    const int iters = 20;
    float ms = timeGemmMs(dA, dB, dC, M, N, K, grid, block, warmup, iters);

    // Optional: GFLOPS
    double gflops = estimateGFLOPS(M, N, K, ms);

    printf("GEMM: M=%d N=%d K=%d | time=%.4f ms | GFLOPS=%.2f\n", M, N, K, ms, gflops);

    // Cleanup
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dC));
    std::free(hA);
    std::free(hB);
    std::free(hC_gpu);
    std::free(hC_ref);

    return ok ? EXIT_SUCCESS : EXIT_FAILURE;
}